## 라이브러리 설치

In [1]:
#언어모델을 활용한 어플리케이션 제작에 도움을 주는 프레임워크
!pip install langchain

#언어모델
!pip install openai

#문장을 벡터로 변환하는데 사용하는 라이브러리
!pip install sentence-transformer

#랭체인에서 제공하는 추가 기능
!pip install -U langchain-community

#비정형 데이터를 처리하고 구조화된 형태로 변환하는 데 사용됨
!pip install unstructured

#nltk 패키지 업데이트되면서 버전충돌때문에 난 에러이므로 3.8.1 < version으로 설치해야 함
!pip install -U nltk>3.9


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement sentence-transformer (from versions: none)
ERROR: No matching distribution found for sentence-transformer

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 9.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... don

In [3]:
import nltk

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /Users/jeon/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [4]:
import nltk
print(nltk.__version__)

3.9.1


In [1]:
#토큰화
!pip install tiktoken

#벡터검색, 유사도 검색
!pip install faiss-cpu


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import openai

import json
import requests


from langchain import LLMChain, OpenAI, PromptTemplate

from langchain.chat_models import ChatOpenAI
#검색을 통해 로드한 문서를 다룰 수 있도록 하는 라이브러리
from langchain.document_loaders import UnstructuredURLLoader
#가져온 텍스트를 가공
from langchain.text_splitter import CharacterTextSplitter

#임베딩, 벡터 스토어
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

#키워드를 검색할 수 있도록 한다.
from langchain.utilities import GoogleSerperAPIWrapper

In [3]:
# 환경변수
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
embeddings = OpenAIEmbeddings()

/var/folders/m1/bqb2x_hs2p3c9xfm8sbxkgcr0000gn/T/ipykernel_71397/2497576997.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [5]:
# test_data

prod_name = "콴첼"

swot_result = {'strengths': ['하루 한 알 복용으로 편리함',
  '알 크기가 작음',
  '가격이 훨씬 착함',
  '개별 포장으로 위생적임',
  '가격이 합리적임'],
 'weaknesses': [],
 'opportunities': ['온라인 구매 시 추가 할인 제공 등 프로모션 전략 개발 가능성',
  '개별 포장된 제품의 위생적 장점을 마케팅 포인트로 활용 가능',
  '건강보조식품 시장에서 젊은 층을 타겟팅한 광고 전략 개발 가능성'],
 'threats': ['기존 브랜드에 대한 충성도 있는 고객층 존재 가능성',
  '경쟁사의 유사한 저렴한 제품 존재 가능성',
  '무료 샘플이나 경쟁사의 프로모션 활동']}

In [6]:
test_oppor = " ".join(swot_result['opportunities'])

In [7]:
test_threat = " ".join(swot_result['threats'])

## 1. 관련 아티클의 리스트를 요청

In [8]:
def search_serp(query):
  #k=몇 개의 리스트를 반환할 것인지
  search = GoogleSerperAPIWrapper(k=5, type="search")

  response_json = search.results(query)
  print(f"Response ====> , {response_json}")

  return response_json

## 2. 잘 맞는 아티클을 찾아서, URL을 리턴하도록 함

In [9]:
#LLM
#결과가 n개 나오도록 하는 template 내용 주의!
def pick_best_articles_urls(response_json, query):
  response_str = json.dumps(response_json)

  llm = ChatOpenAI(temperature=0.5)
  template = """
    너는 소비자 트렌드와 시장 동향을 분석하는 전문 평론가야.
    지금 분석 중인 상품 브랜드의 기회와 위협 요인에 관한 최신 뉴스나 아티클을 찾고 있어.

    QUERY RESPONSE :{response_str}

    위의 리스트는 쿼리 결과에 대한 검색 리스트야 {query}.

    네가 생각하기에 쿼리의 주제를 잘 담고 있는 가장 훌륭한 3개의 아티클을 찾아봐.

    return ONLY an array of urls.
    also make sure the articles are recent and not too old.
    if the file, or URL is invalid, show www.google.com

    """

  prompt_template = PromptTemplate(input_variables=["response_str", "query"], template = template)

  article_chooser_chain = LLMChain(
      llm=llm,
      prompt=prompt_template,
      verbose=True
  )

  urls = article_chooser_chain.run(response_str=response_str, query=query)
  url_list = json.loads(urls)

  print(url_list)
  return url_list

In [10]:
final_query_1 = f'상품명은 {prod_name}, 해당 상품의 {test_oppor}'

In [11]:
final_query_2 = f'상품명은 {prod_name}, 해당 상품의 {test_threat}'

In [12]:
#Google Serper API에서 검색한 내용을 search_result로 받아와 저장(json형태)
search_result = search_serp(final_query_1)

#LLM 템플릿을 통해 가장 잘 맞는 url을 골라, 반환
pick_best_articles_urls(response_json=search_result, query=final_query_1)

Response ====> , {'searchParameters': {'q': '상품명은 콴첼, 해당 상품의 온라인 구매 시 추가 할인 제공 등 프로모션 전략 개발 가능성 개별 포장된 제품의 위생적 장점을 마케팅 포인트로 활용 가능 건강보조식품 시장에서 젊은 층을 타겟팅한 광고 전략 개발 가능성', 'gl': 'us', 'hl': 'en', 'type': 'search', 'num': 5, 'engine': 'google'}, 'organic': [{'title': '상품 프로모션이란? 방법, 전략 및 예시 | 아마존 광고', 'link': 'https://advertising.amazon.com/ko-kr/library/guides/product-promotion', 'snippet': '상품 마케팅 전략의 핵심 요소인 상품 프로모션은 고객에게 상품 가치를 전달하여 매출을 늘리고 브랜드 충성도를 높이는 프로세스입니다. 마케팅 믹스의 4P 중 하나 ...', 'attributes': {'Missing': '상품명 콴첼, 해당 온라인 시 개발 개별 포장 위생적 장점 가능 건강'}, 'position': 1}, {'title': '온라인 스토어를 성장시키고 싶으신가요? 이커머스 전략에 추가할 수 ...', 'link': 'https://www.branchleafdigital.com/ko/want-to-grow-your-online-store-here-are-8-tactics-to-add-to-your-e-commerce-strategy/', 'snippet': '이커머스 비즈니스를 확장하는 것은 쉽지 않은 과정입니다. 이 문서에서는 성장 전략을 시작하는 데 유용한 8가지 전술을 소개합니다.', 'date': 'Jul 10, 2024', 'attributes': {'Missing': '상품명 콴첼, 상품 시 개별 위생적 장점 가능 건강'}, 'rating': 5, 'ratingCount': 10, 'position': 2}, {'title': '2022 프라임데이 

/var/folders/m1/bqb2x_hs2p3c9xfm8sbxkgcr0000gn/T/ipykernel_71397/1683877242.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.5)
/var/folders/m1/bqb2x_hs2p3c9xfm8sbxkgcr0000gn/T/ipykernel_71397/1683877242.py:25: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  article_chooser_chain = LLMChain(
/var/folders/m1/bqb2x_hs2p3c9xfm8sbxkgcr0000gn/T/ipykernel_71397/1683877242.py:31: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  urls = article


> Finished chain.
['https://advertising.amazon.com/ko-kr/library/guides/product-promotion', 'https://www.branchleafdigital.com/ko/want-to-grow-your-online-store-here-are-8-tactics-to-add-to-your-e-commerce-strategy/', 'https://www.sixshop.com/official-blog/%EA%B0%84%EB%8B%A8%ED%95%98%EC%A7%80%EB%A7%8C-%ED%9A%A8%EA%B3%BC%EC%A0%81%EC%9D%B8-%EC%9E%90%EC%82%AC%EB%AA%B0-%EB%A7%88%EC%BC%80%ED%8C%85-%EC%A0%84%EB%9E%B5-5%EA%B0%80%EC%A7%80']


['https://advertising.amazon.com/ko-kr/library/guides/product-promotion',
 'https://www.branchleafdigital.com/ko/want-to-grow-your-online-store-here-are-8-tactics-to-add-to-your-e-commerce-strategy/',
 'https://www.sixshop.com/official-blog/%EA%B0%84%EB%8B%A8%ED%95%98%EC%A7%80%EB%A7%8C-%ED%9A%A8%EA%B3%BC%EC%A0%81%EC%9D%B8-%EC%9E%90%EC%82%AC%EB%AA%B0-%EB%A7%88%EC%BC%80%ED%8C%85-%EC%A0%84%EB%9E%B5-5%EA%B0%80%EC%A7%80']

## URL로 받아온 아티클의 내용 요약하기

In [13]:
def extract_content_from_urls(urls):

  loader = UnstructuredURLLoader(urls=urls)
  data = loader.load()

  text_splitter = CharacterTextSplitter(
      separator='\n',
      chunk_size=1000,
      chunk_overlap=200,
      length_function=len
  )

  docs = text_splitter.split_documents(data)
  db = FAISS.from_documents(docs, embeddings)

  return db

## 요약 진행하기

In [14]:
def summarizer(db, query, k=4):
  docs = db.similarity_search(query, k=k)

  docs_page_content = " ".join([d.page_content for d in docs])

  llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.5)
  template = """
    {docs}

    이 뉴스레터는 이메일로 발성될거야. 스팸처럼 보이지 않도록 아래의 양식을 지켜야 해.

    가이드라인에 맞춰서 적어줘.
    1. 콘텐츠가 밝은 내용이어야 하며, 정보를 얻을 수 있고 유용해야 해.
    2. 콘텐츠가 너무 길지 않은지 확인해야 해.
    3. 콘텐츠는 {query}토픽을 잘 나타내고 있는 내용이어야 해.
    4. 콘텐츠는 읽기 쉽게 쓰여야 하고, 간결해야 해.
    5. 콘텐츠는 독자에게 영감을 줄 수 있어야 해.

    읽기 편하게 한국어로 작성해 줘.

    SUMMARY :
    """

  prompt_template = PromptTemplate(input_variables=["docs", "query"], template=template)

  summarize_chain = LLMChain(llm=llm, prompt=prompt_template, verbose=True)
  response = summarize_chain.run(docs=docs_page_content, query=query)

  return response.replace("\n", "")

In [15]:
query = 'recent famouse K-Pop in global'

search_result = search_serp(query)
urls = pick_best_articles_urls(response_json=search_result, query=query)
data = extract_content_from_urls(urls)
summaries = summarizer(db=data, query=query)

print("URLS : ",  urls)
print("SUMMARIZE : " , summaries)

Response ====> , {'searchParameters': {'q': 'recent famouse K-Pop in global', 'gl': 'us', 'hl': 'en', 'type': 'search', 'num': 5, 'engine': 'google'}, 'organic': [{'title': 'The 10 Most Popular K-Pop Groups Right Now - J-14', 'link': 'https://www.j-14.com/posts/the-10-most-popular-k-pop-groups-right-now/', 'snippet': 'K, Pop OffTop 10 of the Most Popular K-Pop Groups Right Now · BTS · BLACKPINK · Stray Kids · NewJeans · TXT · TWICE · ATEEZ · IVE.', 'date': 'Jul 23, 2024', 'position': 1}, {'title': "2024's Top K-pop Idols - Dance Vision Blog", 'link': 'https://blog.dancevision.com/top-kpop-idols', 'snippet': "2024's Top K-pop Idols · 1. Jungkook (BTS) · 2. Hyunjin (Stray Kids) · 3. Jisoo (BlackPink) · 4. J-Hope (BTS) · 5. Cha Eun Woo (ASTRO) · 6. Minji ( ...", 'date': 'Jan 28, 2024', 'position': 2}, {'title': 'The 2024 Billboard K-Pop Artist 100', 'link': 'https://www.billboard.com/lists/k-pop-artist-100-list-2024-ranked/', 'snippet': "From chart-topping groups like Stray Kids, ATEEZ an

Error fetching or processing https://www.billboard.com/lists/k-pop-artist-100-list-2024-ranked/, exception: Invalid input object: NoneType




> Entering new LLMChain chain...
Prompt after formatting:

    Love J-14? Be sure to subscribe to our YouTube channel for fun, exclusive videos with your favorite stars.
1 of 10
BTS
BTS is widely considered *the* most popular K-pop group globally, and their acclaim is well-deserved. They made history on iTunes by having eight songs reach number one in 100 countries, set a record for the most viewers on a YouTube premiere, and had tremendous success with a U.S. stadium tour.
2 of 10
BLACKPINK
BLACKPINK stands tall as one of today’s most beloved K-pop sensations, and were the very first K-pop group to perform at Coachella… which they did twice! With chart-topping history and electrifying choreography that might just tempt you to learn it yourself, they’re a force to be reckoned with in music.
3 of 10
Stray Kids And despite their humble beginnings, the group soon became internationally loved after they were nominated and won a Billboard Music Award for Top Social Artist in 2017 — the fi

In [17]:
urls

['https://www.j-14.com/posts/the-10-most-popular-k-pop-groups-right-now/',
 'https://blog.dancevision.com/top-kpop-idols',
 'https://www.billboard.com/lists/k-pop-artist-100-list-2024-ranked/']

In [ ]:
query = '글로벌 세계에서 kpop의 위상'

search_result = search_serp(query)
urls = pick_best_articles_urls(response_json=search_result, query=query)
data = extract_content_from_urls(urls)
summaries = summarizer(db=data, query=query)

print("URLS : ",  urls)
print("SUMMARIZE : " , summaries)

Response ====> , {'searchParameters': {'q': '글로벌 세계에서 kpop의 위상', 'gl': 'us', 'hl': 'en', 'type': 'search', 'num': 5, 'engine': 'google'}, 'organic': [{'title': '글로벌 무대 우뚝 선 K-POP, 성공 비결은? - CJ 뉴스룸', 'link': 'https://cjnews.cj.net/%EA%B8%80%EB%A1%9C%EB%B2%8C-%EB%AC%B4%EB%8C%80-%EC%9A%B0%EB%9A%9D-%EC%84%A0-k-pop-%EC%84%B1%EA%B3%B5-%EB%B9%84%EA%B2%B0%EC%9D%80/', 'snippet': '사흘간 무려 14만 명. 미국 현지는 물론 전 세계 각국의 K-POP ... 11년 만에 KPOP과 K컬처의 위상이 눈부시게 높아진 것이다. ... 케이팝이 글로벌에서 이런 ...', 'date': 'Aug 24, 2023', 'position': 1}, {'title': 'K-POP의 글로벌 인기와 세계적 영향력: 음악계를 넘어선 혁신적 영향', 'link': 'https://wisdomagora.com/k-pop%EC%9D%98-%EA%B8%80%EB%A1%9C%EB%B2%8C-%EC%9D%B8%EA%B8%B0%EC%99%80-%EC%84%B8%EA%B3%84%EC%A0%81-%EC%98%81%ED%96%A5%EB%A0%A5-%EC%9D%8C%EC%95%85%EA%B3%84%EB%A5%BC-%EB%84%98%EC%96%B4%EC%84%A0/', 'snippet': '방탄소년단에 이어 라이즈 또한 케이팝을 통하여 한국이라는 나라를 전 세계에 알리고 있다. 케이팝은 하나의 문화산업의 경계에서 벗어나 한국과 다른 나라와의 외교관 그 ...', 'position': 2}, {'title': 'K-POP/해외 - 나무위키', 'link': 'https://namu.wiki/w/K-POP/%ED%95%B4%EC



> Entering new LLMChain chain...
Prompt after formatting:

    글로벌 무대 우뚝 선 K-POP, 성공 비결은?
CULTURE & LIFESTYLE
2023.08.24
페이스북 공유
링크드인 공유
트위터 공유
카카오톡 공유
이메일 공유
링크주소 복사
인쇄
지난 18일부터 20일까지 미국 LA에서 열린 세계 최대 규모 K-컬처 페스티벌 ‘KCON LA 2023’에 역대 최다 관객이 몰렸다. 사흘간 무려 14만 명. 미국 현지는 물론 전 세계 각국의 K-POP팬들이 한데 모여 한국의 음악과 문화를 즐겼다. 온라인에선 176개국 유무〮료 관객 590만 명이 함께했다.
2012년 미국 어바인(Irvine)에서 처음 열린 KCON은 관객 1만 명으로 시작했다. 11년 만에 KPOP과 K컬처의 위상이 눈부시게 높아진 것이다. 글로벌 신드롬의 중심 BTS부터 미국 최대 음악 축제 코첼라의 헤드라이너로 선 블랙핑크, 미국 빌보드영〮국 오피셜 차트에 진입한 4세대 아이돌 뉴진스까지. 변방의 문화였던 K-POP이 글로벌 주류로 발돋움한 비결은 뭘까? 김태훈 팝 칼럼니스트와 김영대 음악평론가의 이야기를 들어봤다.
글로벌 시장 속 K-POP이 걸어온 길
K-POP 산업의 주체는 바로 ‘아이돌’이다. 음악은 기본, 댄스 등 퍼포먼스와 예능프로그램에서의 매력까지 아이돌 아티스트는 그 자체로 하나의 IP다.
우리나라 아이돌의 시초는 1980년대로 거슬러 올라간다. 1988년 서울올림픽 개최를 계기로 세계화에 대한 인식이 재정립됐다. 이전에 우리 음악 안에서 세계적인 동시대성을 구현하는데 만족했지만, 해외 교류가 활발해지면서 글로벌 잠재 소비자에 대한 가능성을 발견했다. TV 매체의 발달로 시각적 요소의 중요성이 커지면서, 트렌드에 맞춰 아티스트를 지속적으로 발전시킬 수 있는 ‘시스템’에 대한 관심도 커졌다. 지난 18일부터 20일까지 미국 LA에서 열린 세계 최대 규모 K-컬처 페스티벌 ‘KCON LA 2023’에 역대 최다 관객이 몰